In [1]:
import sys
import pandas
import numpy
import sklearn
import keras



Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [3]:
# import keras modules

from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [4]:
# import the uci pima indians diabetes dataset
#url = "http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data.csv"
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv(url, names = names)


In [5]:
df.head()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


**Data Preprocessing**

In [7]:
#glucose_concentration
df[df["glucose_concentration"]==0]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


There are 5 instances of glucose_concentration to be null, lets replace all the empty values with NaN and drop them


In [8]:
# define columsn excluding n_pregnant, age and class

columns = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI', 'pedigree_function']

In [9]:
# replace all the zero values with NaN
for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    

In [10]:
#check the dataframe to ensure it worked
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


it worked, now there are no empty values, however the the count in ***serum_insulin*** has reduced from 768 to 394. Now that we know which columsn have rows with missing values, lets drop the missing values.

In [11]:
df.dropna(inplace = True)
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


We dropped all the rows with missing values, now we only have 392 instances in dataset

In [12]:
# convert the dataset in NumPy Array
dataset = df.values
print(dataset.shape)

(392, 9)


In [13]:
# Split the dataset
X = dataset[:, 0:8]
Y = dataset[:, 8].astype(int)

In [14]:
print(X.shape)
print(Y.shape)
print(X[:5])
print (Y[:5])

(392, 8)
(392,)
[[1.000e+00 8.900e+01 6.600e+01 2.300e+01 9.400e+01 2.810e+01 1.670e-01
  2.100e+01]
 [0.000e+00 1.370e+02 4.000e+01 3.500e+01 1.680e+02 4.310e+01 2.288e+00
  3.300e+01]
 [3.000e+00 7.800e+01 5.000e+01 3.200e+01 8.800e+01 3.100e+01 2.480e-01
  2.600e+01]
 [2.000e+00 1.970e+02 7.000e+01 4.500e+01 5.430e+02 3.050e+01 1.580e-01
  5.300e+01]
 [1.000e+00 1.890e+02 6.000e+01 2.300e+01 8.460e+02 3.010e+01 3.980e-01
  5.900e+01]]
[0 1 1 1 1]


In [15]:
#Normalize the dataset

scalar = StandardScaler().fit(X)
print(scalar)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [16]:
# Transfform and stadardize the data

X_standardized = scalar.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


**Building model using Keras**

GridSearchCV -> Grid search , 
KFold        -> Cross Validation ,
Adam -> For Optimizing the model

In [17]:
# Define a random seed
seed = 6
np.random.seed(seed)

def create_model():
    #create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, input_dim = 8, activation='sigmoid'))
    
    #compile model
    adam = Adam(lr = 0.01)
    model.compile(loss = "binary_crossentropy", optimizer = adam, metrics = ['accuracy'])
    
    return model

model = KerasClassifier(build_fn=create_model, verbose = 0)

batch_size = [10, 20, 40]
epochs = [10, 50, 100]

params_grid = dict(batch_size = batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid = params_grid, cv=KFold(random_state=seed), verbose =10 )
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=10 ........................................
Instructions for updating:
Colocations handled automatically by placer.


/Users/kalyanikulkarni/anaconda2/envs/av/lib/python3.7/site-packages/sklearn/model_selection/_split.py:426: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Use tf.cast instead.
[CV]  batch_size=10, epochs=10, score=0.7328244320308889, total=   2.3s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV] . batch_size=10, epochs=10, score=0.75572518492473, total=   1.8s
[CV] batch_size=10, epochs=10 ........................................


...

[CV]  batch_size=10, epochs=100, score=0.8538461419252249, total=   6.6s
[CV] batch_size=20, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   37.5s remaining:    0.0s


[CV]  batch_size=20, epochs=10, score=0.7251908351446836, total=   2.3s
[CV] batch_size=20, epochs=10 ........................................
[CV]  batch_size=20, epochs=10, score=0.7633587890908918, total=   2.5s
[CV] batch_size=20, epochs=10 ........................................
[CV]  batch_size=20, epochs=10, score=0.8153846218035772, total=   2.3s
[CV] batch_size=20, epochs=50 ........................................
[CV]  batch_size=20, epochs=50, score=0.7633587740759813, total=   3.4s
[CV] batch_size=20, epochs=50 ........................................
[CV]  batch_size=20, epochs=50, score=0.7480916007784487, total=   3.5s
[CV] batch_size=20, epochs=50 ........................................
[CV]  batch_size=20, epochs=50, score=0.8076923076923077, total=   3.7s
[CV] batch_size=20, epochs=100 .......................................
[CV]  batch_size=20, epochs=100, score=0.7328244156509865, total=   5.0s
[CV] batch_size=20, epochs=100 ......................................

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.7min finished
/Users/kalyanikulkarni/anaconda2/envs/av/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.801020412876898, using {'batch_size': 40, 'epochs': 50}
0.7653061174312417 (0.03129004898370825) with: {'batch_size': 10, 'epochs': 10}
0.7576530657860697 (0.03366310661221072) with: {'batch_size': 10, 'epochs': 50}
0.7602040807203371 (0.06778175888015583) with: {'batch_size': 10, 'epochs': 100}
0.7678571469625648 (0.03693544362021932) with: {'batch_size': 20, 'epochs': 10}
0.7729591813926794 (0.025249502052574067) with: {'batch_size': 20, 'epochs': 50}
0.762755097631289 (0.03226322829892481) with: {'batch_size': 20, 'epochs': 100}
0.7678571351024569 (0.03997384615499528) with: {'batch_size': 40, 'epochs': 10}
0.801020412876898 (0.027985692234937354) with: {'batch_size': 40, 'epochs': 50}
0.7882653135730295 (0.02466454255723237) with: {'batch_size': 40, 'epochs': 100}


In [ ]:
#model = create_model()
#print(model.summary())

**Perform GridSearch**

1. **1. Best: 0.7857142913402343, using {'batch_size': 20, 'epochs': 50}**
1. 1. 0.7831632592240159 (0.03944901412222824) with: {'batch_size': 10, 'epochs': 10}
1. 1. 0.7602040807203371 (0.02481520938164371) with: {'batch_size': 10, 'epochs': 50}
1. 1. 0.7678571437694588 (0.038896956191863556) with: {'batch_size': 10, 'epochs': 100}
1. 1. 0.7627551094913969 (0.0367762267910484) with: {'batch_size': 20, 'epochs': 10}
1. 1. 0.7857142913402343 (0.024343215526919076) with: {'batch_size': 20, 'epochs': 50}
1. 1. 0.7806122459623278 (0.035877667051783815) with: {'batch_size': 20, 'epochs': 100}
1. 1. 0.7653061125655564 (0.03699373723334932) with: {'batch_size': 40, 'epochs': 10}
1. 1. 0.785714290123813 (0.021255199793276124) with: {'batch_size': 40, 'epochs': 50}
1. 1. 0.7678571437694588 (0.045293374010821584) with: {'batch_size': 40, 'epochs': 100}

**Best: 0.801020412876898, using {'batch_size': 40, 'epochs': 50}**
0.7653061174312417 (0.03129004898370825) with: {'batch_size': 10, 'epochs': 10}
0.7576530657860697 (0.03366310661221072) with: {'batch_size': 10, 'epochs': 50}
0.7602040807203371 (0.06778175888015583) with: {'batch_size': 10, 'epochs': 100}
0.7678571469625648 (0.03693544362021932) with: {'batch_size': 20, 'epochs': 10}
0.7729591813926794 (0.025249502052574067) with: {'batch_size': 20, 'epochs': 50}
0.762755097631289 (0.03226322829892481) with: {'batch_size': 20, 'epochs': 100}
0.7678571351024569 (0.03997384615499528) with: {'batch_size': 40, 'epochs': 10}
0.801020412876898 (0.027985692234937354) with: {'batch_size': 40, 'epochs': 50}
0.7882653135730295 (0.02466454255723237) with: {'batch_size': 40, 'epochs': 100}


Let's set hardcoded **Epoch** and **Batch_size** value and do one more round of grid search to find the learning rate

In [ ]:
GridSearchCV??

In [18]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)


# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 50, batch_size = 40, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  dropout_rate=0.0, learn_rate=0.001, score=0.7099236559321862, total=   4.2s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s


[CV]  dropout_rate=0.0, learn_rate=0.001, score=0.7633587809009407, total=   4.1s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


...

[CV]  dropout_rate=0.0, learn_rate=0.1, score=0.7615384642894452, total=   4.8s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   40.2s remaining:    0.0s


[CV]  dropout_rate=0.1, learn_rate=0.001, score=0.7328244156509865, total=   5.3s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV]  dropout_rate=0.1, learn_rate=0.001, score=0.7557251903846973, total=   5.4s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV]  dropout_rate=0.1, learn_rate=0.001, score=0.7923076794697688, total=   5.7s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV]  dropout_rate=0.1, learn_rate=0.01, score=0.7557251853797272, total=   5.6s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV]  dropout_rate=0.1, learn_rate=0.01, score=0.7862595588196325, total=   5.7s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV]  dropout_rate=0.1, learn_rate=0.01, score=0.8230769175749558, total=   5.9s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV]  dropout_rate=0.1, learn_rate=0.1, score=0.740458014357181, total=   6.6s
[CV] d

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.6min finished
/Users/kalyanikulkarni/anaconda2/envs/av/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.7882653080991336, using {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7602040781354418 (0.039950692194583824) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7653061274667176 (0.03741848921440294) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7448979669383594 (0.015000886711784485) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.7602040732697565 (0.024474835613549094) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7882653080991336 (0.027515203721844652) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.760204090299655 (0.01829285451845613) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.7678571320614036 (0.03476261152807619) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7780612313321659 (0.042546149635136865) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7806122414007479 (0.0524286484370494) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


**Best: 0.7933673562139881, using {'dropout_rate': 0.0, 'learn_rate': 0.001}**

In [19]:


# Define a random seed
seed = 6
np.random.seed(seed)


# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 50, batch_size = 40, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear' ]
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation=activation, init=init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  activation=softmax, init=uniform, score=0.7099236741320778, total=   8.5s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.6s remaining:    0.0s


[CV]  activation=softmax, init=uniform, score=0.7862595488096922, total=   7.6s
[CV] activation=softmax, init=uniform ................................


...

[CV]  activation=softmax, init=zero, score=0.7923077023946322, total=   8.6s
[CV] activation=relu, init=uniform ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.3min remaining:    0.0s


[CV]  activation=relu, init=uniform, score=0.7099236741320778, total=   9.0s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.7633587708910003, total=   9.0s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.8461538369839008, total=   8.8s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.7251908351446836, total=   9.1s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.7862595588196325, total=   9.3s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.8153846172186044, total=   9.4s
[CV] activation=relu, init=zero ......................................
[CV]  activation=relu, init=zero, score=0.6946564867296292, total=   9.5s
[CV] activation=relu, init=zero .........

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  6.0min finished
/Users/kalyanikulkarni/anaconda2/envs/av/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.7908163300278236, using {'activation': 'linear', 'init': 'uniform'}
0.7678571500036181 (0.04196469069516574) with: {'activation': 'softmax', 'init': 'uniform'}
0.7755102045377906 (0.018344828537776046) with: {'activation': 'softmax', 'init': 'normal'}
0.7780612282911126 (0.021242139904587655) with: {'activation': 'softmax', 'init': 'zero'}
0.7729591813926794 (0.05599445876021457) with: {'activation': 'relu', 'init': 'uniform'}
0.7755102087952652 (0.037577468002655524) with: {'activation': 'relu', 'init': 'normal'}
0.7704081652419907 (0.058958442876489346) with: {'activation': 'relu', 'init': 'zero'}
0.7857142913402343 (0.02181171220743405) with: {'activation': 'tanh', 'init': 'uniform'}
0.783163267282807 (0.030761285035728857) with: {'activation': 'tanh', 'init': 'normal'}
0.7882653108360816 (0.045547118372335466) with: {'activation': 'tanh', 'init': 'zero'}
0.7908163300278236 (0.04324297240305246) with: {'activation': 'linear', 'init': 'uniform'}
0.7678571378394049 (0.04085047

**Best: 0.7908163300278236, using {'activation': 'linear', 'init': 'uniform'}**

In [23]:
# Define a random seed
seed = 6
np.random.seed(seed)


# Start defining the model
def create_model(neuron1, neuron2):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer='uniform', activation='linear'))
    model.add(Dropout(0.1))
    model.add(Dense(neuron2, input_dim = 8, kernel_initializer='uniform', activation='linear'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 50, batch_size = 40, verbose = 0)

# define the grid search parameters
neuron1 = [4, 8, 16 ]
neuron2 = [2, 4, 8]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1=neuron1, neuron2=neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ... neuron1=4, neuron2=2, score=0.7633587658860301, total=  13.6s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.6s remaining:    0.0s


[CV] ... neuron1=4, neuron2=2, score=0.7633587809009407, total=  16.0s
[CV] neuron1=4, neuron2=2 ............................................


...

[CV] ... neuron1=4, neuron2=8, score=0.8461538369839008, total=  12.9s
[CV] neuron1=8, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.3min remaining:    0.0s


[CV] ... neuron1=8, neuron2=2, score=0.7709923645922245, total=  13.0s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ... neuron1=8, neuron2=2, score=0.7633587809009407, total=  13.3s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ... neuron1=8, neuron2=2, score=0.8384615366275494, total=  13.2s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ... neuron1=8, neuron2=4, score=0.7709923645922245, total=  13.3s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ... neuron1=8, neuron2=4, score=0.7633587809009407, total=  13.5s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ... neuron1=8, neuron2=4, score=0.8307692179313073, total=  16.5s
[CV] neuron1=8, neuron2=8 ............................................
[CV] ... neuron1=8, neuron2=8, score=0.7557251853797272, total=  16.0s
[CV] neuron1=8, neuron2=8 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  6.7min finished


Best: 0.7908163260744543, using {'neuron1': 4, 'neuron2': 4}
0.788265302017027 (0.03535836469806013) with: {'neuron1': 4, 'neuron2': 2}
0.7908163260744543 (0.028142939016169974) with: {'neuron1': 4, 'neuron2': 4}
0.7908163260744543 (0.038979893758981324) with: {'neuron1': 4, 'neuron2': 8}
0.7908163260744543 (0.03370616232395781) with: {'neuron1': 8, 'neuron2': 2}
0.788265302017027 (0.030102039602569602) with: {'neuron1': 8, 'neuron2': 4}
0.788265302017027 (0.04089605834313712) with: {'neuron1': 8, 'neuron2': 8}
0.7831632660663858 (0.03367864056080986) with: {'neuron1': 16, 'neuron2': 2}
0.785714290123813 (0.03173682706863293) with: {'neuron1': 16, 'neuron2': 4}
0.7831632660663858 (0.0394490161236312) with: {'neuron1': 16, 'neuron2': 8}


**Best: 0.7908163260744543, using {'neuron1': 4, 'neuron2': 4}**

In [24]:
Y_pred = grid.predict(X_standardized)

In [25]:
print(Y_pred.shape)

(392, 1)


In [26]:
print(Y_pred[:5])

[[0]
 [1]
 [0]
 [1]
 [1]]


In [27]:
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, Y_pred))
print(classification_report(Y, Y_pred))

0.7882653061224489
              precision    recall  f1-score   support

           0       0.81      0.90      0.85       262
           1       0.74      0.56      0.64       130

   micro avg       0.79      0.79      0.79       392
   macro avg       0.77      0.73      0.74       392
weighted avg       0.78      0.79      0.78       392

